In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
# Parameters
n_steps = 10000
dt = 0.1
ref_price_a, ref_price_b = 100, 100
lambda_a, lambda_b = 1.0, 1.0
std_price_a, std_price_b = 1.0, 1.0
delta_a, delta_b = dt * 0.01, dt * 0.01
phi_a, phi_b = dt * 15, dt * 15
kappa = dt * 0.01
initial_particles_a = 100
initial_particles_b = 100
n_histograms = 5000

# Initialize particles
prices_a = np.random.normal(loc=ref_price_a, scale=std_price_a, size=initial_particles_a)
sizes_a = np.random.exponential(scale=1 / lambda_a, size=initial_particles_a)
prices_b = np.random.normal(loc=ref_price_b, scale=std_price_b, size=initial_particles_b)
sizes_b = np.random.exponential(scale=1 / lambda_b, size=initial_particles_b)

particles_a = np.column_stack((prices_a, sizes_a))
particles_b = np.column_stack((prices_b, sizes_b))

starting_distribution_a = particles_a.copy()
starting_distribution_b = particles_b.copy()

price_bins = np.linspace(
    min(ref_price_a, ref_price_b) - 5 * std_price_b,
    max(ref_price_a, ref_price_b) + 5 * std_price_a,
    500
)

dp = price_bins[1]-price_bins[0]

# Arrays to store statistics
time_steps = []
n_a_array = []
n_b_array = []
mean_price_a_array = []
mean_price_b_array = []
std_price_a_array = []
std_price_b_array = []

# To store histograms of the last n steps
last_histograms_a = []
last_histograms_b = []

# Processes
def add_entries():
    global particles_a, particles_b

    n_new_a = np.random.poisson(phi_a)
    n_new_b = np.random.poisson(phi_b)

    if n_new_a > 0:
        new_prices_a = np.random.normal(loc=ref_price_a, scale=std_price_a, size=n_new_a)
        new_sizes_a = np.random.exponential(scale=1 / lambda_a, size=n_new_a)
        particles_a = np.vstack((particles_a, np.column_stack((new_prices_a, new_sizes_a))))

    if n_new_b > 0:
        new_prices_b = np.random.normal(loc=ref_price_b, scale=std_price_b, size=n_new_b)
        new_sizes_b = np.random.exponential(scale=1 / lambda_b, size=n_new_b)
        particles_b = np.vstack((particles_b, np.column_stack((new_prices_b, new_sizes_b))))

def perform_interactions():
    global particles_a, particles_b

    def process_interactions(particles_x, particles_y, kappa):

        interactions_rate = kappa * len(particles_y)

        for i in range(len(particles_x)):
            n_interactions = np.random.poisson(interactions_rate)

            if len(particles_y) == 0:
                break
            if n_interactions > 0:
                chosen_indices = np.random.choice(len(particles_y), size=min(n_interactions, len(particles_y)), replace=False)

                for idx in chosen_indices:
                    price_x, size_x = particles_x[i]
                    price_y, size_y = particles_y[idx]

                    # Uncomment for volumes

                    # if price_y >= price_x:
                    #     if size_x < size_y:
                    #         particles_y[idx, 1] -= size_x
                    #         particles_x[i] = [0, 0]  # Marked for deletion
                    #         break
                    #     if size_x > size_y:
                    #         particles_x[i, 1] -= size_y

                    #         if particles_x[i, 1] < 0:
                    #             print('Negative volume')

                    #         particles_y[idx] = [0, 0]  # Marked for deletion


                    # Uncomment for no volumes        

                    if price_y >= price_x:
                        particles_x[i] = [0, 0]  # Marked for deletion
                        particles_y[idx] = [0, 0]  # Marked for deletion
                        break

            particles_y = particles_y[np.any(particles_y != [0, 0], axis=1)]
        particles_x = particles_x[np.any(particles_x != [0, 0], axis=1)]
        
        return particles_x, particles_y

    particles_a, particles_b = process_interactions(particles_a, particles_b, kappa)
    
def apply_cancellations():
    global particles_a, particles_b
    particles_a = particles_a[np.random.rand(len(particles_a)) > delta_a]
    particles_b = particles_b[np.random.rand(len(particles_b)) > delta_b]

def record_totals(step):
    global particles_a, particles_b, time_steps

    time_steps.append(step)
    n_a_array.append(len(particles_a))
    n_b_array.append(len(particles_b))

def record_histograms(n_histograms = n_histograms):
    global last_histograms_a, last_histograms_b
    
    hist_a, _ = np.histogram(particles_a[:, 0], bins=price_bins)
    hist_b, _ = np.histogram(particles_b[:, 0], bins=price_bins)
    
    last_histograms_a.append(hist_a)
    last_histograms_b.append(hist_b)

    if len(last_histograms_a) > n_histograms:
        last_histograms_a.pop(0)
    if len(last_histograms_b) > n_histograms:
        last_histograms_b.pop(0)

def one_step(step):
    record_totals(step)
    add_entries()
    perform_interactions()
    apply_cancellations()
    
    if step >= n_steps - n_histograms:
        record_histograms()

# Run simulation
for step in range(n_steps):
    one_step(step)
    if len(particles_a) == 0 and len(particles_b) == 0:
        print(f"Simulation ended at step {step}.")
        break

# Average histograms
final_counts_a = np.mean(last_histograms_a/dp, axis=0)
final_counts_b = np.mean(last_histograms_b/dp, axis=0)

initial_counts_a, _ = np.histogram(starting_distribution_a[:, 0], bins=price_bins)
initial_counts_b, _ = np.histogram(starting_distribution_b[:, 0], bins=price_bins)
xvalues = [(price_bins[i] + price_bins[i+1])/2 for i in range(len(price_bins) -1 )]

In [ ]:
with open("resources\\Simulacion mercado informal\\totales_n_a_10.pkl", 'wb') as f:
    pickle.dump(n_a_array,f)
with open("resources\\Simulacion mercado informal\\totales_n_b_10.pkl", 'wb') as f:
    pickle.dump(n_b_array,f)
with open("resources\\Simulacion mercado informal\\times_10.pkl", 'wb') as f:
    pickle.dump(time_steps,f)
with open("resources\\Simulacion mercado informal\\n_A_estacionario_10.pkl", 'wb') as f:
    pickle.dump(final_counts_a,f)
with open("resources\\Simulacion mercado informal\\n_B_estacionario_10.pkl", 'wb') as f:
    pickle.dump(final_counts_b,f)